In [1]:
import requests
from PIL import Image
from transformers import AutoTokenizer, ViTImageProcessor, VisionEncoderDecoderModel
from transformers import pipeline


In [3]:
# 各種モデルの読み込み
# model = VisionEncoderDecoderModel.from_pretrained("./models/vit-gpt2-japanese-image-captioning_stair-captions/checkpoint-613500/")
# model = VisionEncoderDecoderModel.from_pretrained("./models/vit-gpt2-japanese-image-captioning_stair-captions-result")
model = VisionEncoderDecoderModel.from_pretrained("fujie/fujie_vit-gpt2-japanese-image-captioning_stair-captions-result")
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium", use_fast=False)
tokenizer.do_lower_case = True  # due to some bug of tokenizer config loading
image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

In [4]:
# let's perform inference on an image
url = "http://images.cocodataset.org/val2017/000000039769.jpg"

image = Image.open(requests.get(url, stream=True).raw)
pixel_values = image_processor(image, return_tensors="pt").pixel_values

# autoregressively generate caption (uses greedy decoding by default)
generated_ids = model.generate(pixel_values,
                               max_new_tokens=30, 
                               num_beams=5,
                               early_stopping=True,
                               do_sample=True,
                               temperature=1.2,
                               top_k=50,
                               top_p=0.95, 
                               no_repeat_ngram_size=3,
                               num_return_sequences=5)
generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
for text in generated_texts:
    print(text)

ピンクの布の上に二匹の猫がいる
ピンクの布の上に二匹の猫が寝ている
2匹の猫がピンクのクッションの上でじゃれている
2匹の猫がピンクのクッションで遊んでいる
ピンクの布の上に二匹の猫が寝転んでいる


In [5]:
# model.save_pretrained("./models/vit-gpt2-japanese-image-captioning_stair-captions-result/")

In [6]:
pl = pipeline("image-to-text",
             model=model,
             tokenizer=tokenizer,
             feature_extractor=image_processor,)

In [7]:
pl(url)

/home/fujie/.conda/envs/py311/lib/python3.11/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'ピンクの布の上に二匹の猫がいる'}]

In [8]:
pl.save_pretrained("./models/vit-gpt2-japanese-image-captioning_stair-captions-result-pipeline/")

In [8]:
# pl("./photos/006.jpg")